In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [2]:
cap = cv2.VideoCapture(0)

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

_, prev = cap.read()
prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
prev_pts = cv2.goodFeaturesToTrack(prev_gray,
                                    maxCorners=200,
                                    qualityLevel=0.01,
                                    minDistance=30,
                                    blockSize=3)
while(1):
    ret, frame = cap.read()
    if not ret: continue
    curr_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    curr_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_pts, None) 
        
    idx = np.where(status==1)[0]
    prev_pts = prev_pts[idx]
    curr_pts = curr_pts[idx]
    
    m, inliners = cv2.estimateAffinePartial2D(prev_pts, curr_pts, False)
    
    res = cv2.warpAffine(curr_gray, cv2.UMat(m), (w, h), flags=cv2.INTER_NEAREST|cv2.WARP_INVERSE_MAP)
    
    cv2.imshow('frame', res)

    if cv2.waitKey(3) & 0xff == 27:
        break
    
cap.release()
cv2.destroyAllWindows()